# Postgres Visual Semantic Knowledge Graph (Cloud SQL)


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/billyjacobson/cloud-sql-pgvector-graph/blob/main/cloud-sql-pgvector-graph.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fbillyjacobson%2Fcloud-sql-pgvector-graph%2Fmain%2Fcloud-sql-pgvector-graph.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/billyjacobson/cloud-sql-pgvector-graph/main/cloud-sql-pgvector-graph.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/billyjacobson/cloud-sql-pgvector-graph/blob/main/cloud-sql-pgvector-graph.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

In this tutorial, you'll visualize facts about Space Exploration and Astrophysics as a semantic knowledge graph using Google Cloud SQL (PostgreSQL), `pgvector`, and `pyvis`.

The steps performed include:

*   Configure a Google Cloud SQL (PostgreSQL) instance.
*   Enable the `pgvector` extension for vector similarity search.
*   Generate semantic embeddings for text data using `sentence-transformers`.
*   Store and query vector data in PostgreSQL.
*   Visualize the relationships between data points using `pyvis`.

## Costs

This tutorial uses billable components of Google Cloud:

*   **Cloud SQL**: Costs are associated with the instance size and storage used.

Learn about [Cloud SQL pricing](https://cloud.google.com/sql/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## 1. Setup and Installation

In [ ]:
!pip uninstall -y protobuf
# Install specific protobuf version and other dependencies
!pip install protobuf==3.20.3 "cloud-sql-python-connector[pg8000]" pgvector sentence-transformers pyvis

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## 2. Authentication and Configuration

Please fill in your Cloud SQL instance details below.

In [ ]:
# Configuration - REPLACE THESE WITH YOUR VALUES
INSTANCE_CONNECTION_NAME = "project:region:instance"  # e.g., my-project:us-central1:my-instance
DB_USER = "postgres"
DB_PASS = "password"
DB_NAME = "postgres"

## 3. Database Connection

In [ ]:
from google.cloud.sql.connector import Connector, IPTypes
import pg8000
import sqlalchemy
from sqlalchemy import create_engine, text

# Initialize Connector
connector = Connector()

def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
        ip_type=IPTypes.PUBLIC  # Adjust if using Private IP
    )
    return conn

# Create connection pool
pool = create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

print("Database connection pool created.")

## 4. Database Schema Setup

Enables `pgvector` extension and creates the `space_facts` table.

In [ ]:
with pool.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS space_facts (
            id SERIAL PRIMARY KEY,
            fact_text TEXT,
            category TEXT,
            embedding VECTOR(384)
        );
    """))
    conn.commit()
    print("Table 'space_facts' ready and vector extension enabled.")

## 5. Data Generation and Vectorization

In [ ]:
from sentence_transformers import SentenceTransformer
import pgvector.sqlalchemy

# 1. Define Facts
facts_data = [
    ("Apollo Missions", "Apollo 11 was the first mission to land humans on the Moon."),
    ("Apollo Missions", "Neil Armstrong was the first person to walk on the lunar surface."),
    ("Apollo Missions", "The Apollo 13 mission is known as a 'successful failure' due to the safe return of the crew after an explosion."),
    ("Apollo Missions", "Apollo 8 was the first crewed spacecraft to leave low Earth orbit and orbit the Moon."),
    ("Apollo Missions", "A total of 12 astronauts have walked on the Moon."),
    ("Apollo Missions", "The Saturn V rocket remains the tallest, heaviest, and most powerful rocket ever brought to operational status."),
    ("Apollo Missions", "Apollo 17 was the final mission of NASA's Apollo program."),
    ("Apollo Missions", "Harrison Schmitt was the first and only scientist (geologist) to walk on the Moon during Apollo 17."),
    ("Apollo Missions", "The Lunar Module was designed to fly only in the vacuum of space."),
    ("Apollo Missions", "Apollo computers had less processing power than a modern pocket calculator."),

    ("Exoplanets", "Proxima Centauri b is the closest known exoplanet to our Solar System."),
    ("Exoplanets", "51 Pegasi b was the first exoplanet discovered orbiting a main-sequence star."),
    ("Exoplanets", "The Kepler Space Telescope discovered thousands of exoplanets during its mission."),
    ("Exoplanets", "'Hot Jupiters' are gas giant exoplanets that orbit very close to their host stars."),
    ("Exoplanets", "The TRAPPIST-1 system contains seven Earth-sized planets."),
    ("Exoplanets", "Super-Earths are planets with a mass higher than Earth's but substantially below that of the Solar System's ice giants."),
    ("Exoplanets", "Wasp-76b rains iron on its night side due to extreme temperatures."),
    ("Exoplanets", "Some exoplanets are tidally locked, meaning one side always faces the star."),
    ("Exoplanets", "The Goldilocks Zone is the habitable zone where liquid water can exist on a planet's surface."),
    ("Exoplanets", "Rogue planets drift through space without orbiting a star."),

    ("Black Holes", "Sagittarius A* is the supermassive black hole at the center of the Milky Way."),
    ("Black Holes", "The event horizon is the boundary around a black hole beyond which nothing can escape."),
    ("Black Holes", "Black holes can form from the collapse of massive stars."),
    ("Black Holes", "Spaghettification is the vertical stretching and horizontal compression of objects in a strong gravitational field."),
    ("Black Holes", "Stephen Hawking predicted that black holes emit radiation, now known as Hawking radiation."),
    ("Black Holes", "The first image of a black hole was captured by the Event Horizon Telescope in 2019."),
    ("Black Holes", "Stellar-mass black holes are typically 5 to 10 times the mass of the Sun."),
    ("Black Holes", "Quasars are extremely luminous active galactic nuclei powered by supermassive black holes."),
    ("Black Holes", "Time slows down significantly near a black hole due to gravitational time dilation."),
    ("Black Holes", "Two colliding black holes can send ripples through spacetime called gravitational waves."),

    ("Mars Exploration", "The Viking 1 lander was the first spacecraft to successfully land on Mars and complete its mission."),
    ("Mars Exploration", "Olympus Mons on Mars is the tallest volcano in the Solar System."),
    ("Mars Exploration", "The Curiosity rover landed on Mars in 2012 using a 'sky crane' maneuver."),
    ("Mars Exploration", "Mars is known as the Red Planet due to iron oxide on its surface."),
    ("Mars Exploration", "Valles Marineris is a canyon system on Mars that dwarfs the Grand Canyon."),
    ("Mars Exploration", "The Perseverance rover carries a small helicopter named Ingenuity."),
    ("Mars Exploration", "Liquid water cannot exist on the surface of Mars due to low atmospheric pressure."),
    ("Mars Exploration", "A Martian day is called a 'sol' and is slightly longer than an Earth day."),
    ("Mars Exploration", "Dust storms on Mars can cover the entire planet and last for months."),
    ("Mars Exploration", "SpaceX's Starship is being designed with the goal of carrying humans to Mars."),

    ("Cosmology", "The Big Bang theory explains the expansion of the universe from a high-density state."),
    ("Cosmology", "Dark energy is a mysterious force driving the accelerated expansion of the universe."),
    ("Cosmology", "Dark matter accounts for about 85% of the matter in the universe."),
    ("Cosmology", "The Cosmic Microwave Background is the afterglow radiation from the Big Bang."),
    ("Cosmology", "The universe is estimated to be about 13.8 billion years old."),
    ("Solar System", "The Voyager 1 probe is the most distant human-made object in space."),
    ("Solar System", "Jupiter's Great Red Spot is a giant storm that has raged for centuries."),
    ("Solar System", "Venus is the hottest planet in the Solar System due to a runaway greenhouse effect."),
    ("Solar System", "The Sun contains 99.86% of the mass in the Solar System."),
    ("Solar System", "The Kuiper Belt is a region of icy bodies beyond the orbit of Neptune.")
]

# 2. Load Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Embed and Insert
with pool.connect() as conn:
    # Clear existing data to avoid duplicates on re-run
    conn.execute(text("TRUNCATE TABLE space_facts RESTART IDENTITY;"))

    print("Generating embeddings and inserting data...")
    for category, fact in facts_data:
        embedding = model.encode(fact).tolist()
        conn.execute(
            text("INSERT INTO space_facts (fact_text, category, embedding) VALUES (:text, :cat, :emb)"),
            {"text": fact, "cat": category, "emb": str(embedding)}
        )
    conn.commit()
    print("Successfully inserted 50 facts.")

## 6. Build the Knowledge Graph

Setup the options for the network visualization.

In [ ]:

from pyvis.network import Network

# 1. Initialize Midnight Theme
net = Network(
    height="750px",
    width="100%",
    notebook=True,
    cdn_resources='in_line',
    bgcolor="#0b0e14",
    font_color="#f8f9fa"
)

net.set_options("""
{
  "nodes": {
    "font": {
        "size": 11,
        "face": "Playfair Display, serif",
        "color": "#f8f9fa"
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -4000,
      "centralGravity": 0.6,
      "springLength": 90,
      "springConstant": 0.03,
      "damping": 0.4,
      "avoidOverlap": 0.7
    },
    "maxVelocity": 12,
    "minVelocity": 0.1,
    "stabilization": {
      "enabled": true,
      "iterations": 1500
    }
  },
  "interaction": {
    "hover": true,
    "tooltipDelay": 150
  }
}
""")

colors = {
    "Apollo Missions": "#d4af37",
    "Exoplanets": "#e5989b",
    "Black Holes": "#00f5ff",
    "Mars Exploration": "#ff4d6d",
    "Cosmology": "#b8c0ff",
    "Solar System": "#9d4edd"
}

We will query the nearest neighbors for each fact using the Cosine Distance operator `<=>`.

In [ ]:
from sqlalchemy import text
import pandas as pd
import IPython

with pool.connect() as conn:
    result = conn.execute(text("SELECT id, fact_text, category, embedding FROM space_facts ORDER BY id"))
    all_facts = result.fetchall()

    unique_edges = {}
    node_connection_count = {row.id: 0 for row in all_facts}

    # Query for similar facts.
    for row in all_facts:
        query = text("""
            SELECT id, 1 - (embedding <=> :emb) as similarity
            FROM space_facts
            WHERE id != :id
            ORDER BY embedding <=> :emb ASC
            LIMIT 6  -- Increased limit to allow more potential connections
        """)
        neighbors = conn.execute(query, {"emb": str(row.embedding), "id": row.id}).fetchall()

        for neighbor in neighbors:
            if neighbor.similarity > 0.35:
                edge_pair = tuple(sorted((row.id, neighbor.id)))
                if edge_pair not in unique_edges:
                    thickness = (neighbor.similarity ** 3) * 25
                    unique_edges[edge_pair] = thickness
                    node_connection_count[row.id] += 1
                    node_connection_count[neighbor.id] += 1

    # Build the node network.
    for row in all_facts:
        short_label = " ".join(row.fact_text.split()[:3]).upper()
        # Scale nodes by connection count so hubs stand out.
        node_size = 10 + (node_connection_count[row.id] * 4)

        net.add_node(
            row.id,
            label=short_label,
            title=row.fact_text,
            color=colors.get(row.category, "#495057"),
            size=node_size,
            borderWidth=0,
            shadow={"enabled": True, "color": "rgba(255,255,255,0.1)", "size": 12}
        )

    # Add Edges with opacity logic.
    for (src, dst), width in unique_edges.items():
        alpha = min(0.4, (width / 25))
        net.add_edge(src, dst, width=width, color=f"rgba(255,255,255,{alpha})")

# Save and Display
net.save_graph("knowledge_graph.html")
html_content = open("knowledge_graph.html", "r").read()
font_link = '<link href="https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700&display=swap" rel="stylesheet">'
html_content = html_content.replace('<head>', '<head>' + font_link)

with open("knowledge_graph.html", "w") as f:
    f.write(html_content)

IPython.display.HTML(filename="knowledge_graph.html")